We process the MAT files to NPY files. In addition, heavy data augmentation is performed.

In [ ]:
import os 
import os.path as path

import numpy as np
import scipy.io as sio

from imgaug import augmenters as iaa

In [ ]:
data_dir = 'NPY_FILES'
source_dir = 'MAT_FILES'
use_deconv = 'D_'
# use_deconv = ''

if use_deconv: 
    data_dir = use_deconv + data_dir

try:
    os.listdir(data_dir)
except:
    os.mkdir(data_dir)

In [ ]:
class ImageAugmentor:
    def __init__(self, images, mode='SIMPLE', labels=None, end_to_end=True):
        self._source_images = images
        self._source_labels = labels
        self._mode = mode
        self._result_images = [self._source_images.copy()]
        self._result_labels = [self._source_labels.copy()]
        self._end_to_end = end_to_end

    def generate(self):
        if self._mode == 'SIMPLE':
            self._geometry_augment()
        else:
            raise NotImplementedError
            
        if self._end_to_end:
            return np.vstack(self._result_images), np.vstack(
                self._result_labels)
        else:
            return self._result_images, self._result_labels

    def _geometry_augment(self, dropout_prob=0.15):
        # rotation, by 
        rot_augs = [iaa.Affine(rotate=(deg)) for deg in range(0, 90, 15)]
        for aug in rot_augs:
            self._result_images.append(aug.augment_images(self._source_images))
            self._result_labels.append(aug.augment_images(self._source_labels))
            
        self._source_images = np.vstack(self._result_images.copy())
        self._source_labels = np.vstack(self._result_labels.copy())
        # for those need to modify the mask as well
        flip_lr = iaa.Fliplr(1)
        flip_ud = iaa.Flipud(1)
        flip_lrud = iaa.Sequential([flip_lr, flip_ud])
        mask_modified_augs = [flip_lr, flip_ud, flip_lrud]
        for aug in mask_modified_augs:
            self._result_images.append(aug.augment_images(self._source_images))
            self._result_labels.append(aug.augment_images(self._source_labels))

        # NOTE: directly convert all images to whole black without modify the mask
        # This is caused by the `imgaug` implementation
        # But we actually got benefited... by super strong regularization
        dropout = iaa.Dropout(p=dropout_prob)
        mask_no_modified_augs = [dropout]
        length = len(self._result_images)
        for aug in mask_no_modified_augs:
            for grp_idx in range(length):
                self._result_images.append(
                    aug.augment_images(np.array((self._result_images[grp_idx])).astype(
                        np.uint8)))
                self._result_labels.append(self._result_labels[grp_idx])

In [ ]:
size = [128] # 256, 512
dataset = ['train', 'test']

train_size = 19
test_size = 58

for s in size:
    for ds in dataset:
        key = '{}{}x{}'.format(ds, s, s)
        fname = '{}.mat'.format(key)
        data = sio.loadmat(path.join(source_dir, fname))[key]
        print(fname)
        switch_dim = lambda x: np.rollaxis(x, -1, 0)
        images = switch_dim(data[0][0][0])
        
        if use_deconv:
            key_ = '{}{}x{}'.format(ds, 512, 512)
            fname_ = '{}.mat'.format(key_)
            print(fname_)
            data_ = sio.loadmat(path.join(source_dir, fname_))[key_]
        else:
            key_ = key
            fname_ = fname
            print(fname_)
            data_ = data
        lumen = switch_dim(data_[0][0][1])
        media = switch_dim(data_[0][0][2])

        raw_images = images
        raw_lumen = lumen
        raw_media = media
        
        print(raw_images.shape)

        aug_images = []
        aug_lumens = []
        aug_medias = []
        
        if ds == 'train':
            print('Generating...')
            ia = ImageAugmentor(raw_images, labels=raw_lumen, mode='SIMPLE')
            aug_images, aug_lumen = ia.generate()
            ia = ImageAugmentor(raw_images, labels=raw_media, mode='SIMPLE')
            aug_images, aug_media = ia.generate()
            print(aug_images.shape, aug_lumen.shape, aug_media.shape)
            print('Saving training data...')
            np.save(
                path.join(
                    path.abspath('.'), data_dir,
                    'aug_train_data_{}.npy'.format(s)), aug_images)
            np.save(
                path.join(
                    path.abspath('.'),
                    data_dir, 'aug_train_lumen_labels_{}.npy'.format(s)), aug_lumen)
            np.save(
                path.join(
                    path.abspath('.'),
                    data_dir, 'aug_train_media_labels_{}.npy'.format(s)), aug_media)
        else:
            print('Saving test data...')
            np.save(
                path.join(
                    path.abspath('.'), data_dir,
                    'test_data_{}.npy'.format(s)), raw_images)
            np.save(
                path.join(
                    path.abspath('.'),
                    data_dir, 'test_lumen_labels_{}.npy'.format(s)), raw_lumen)
            np.save(
                path.join(
                    path.abspath('.'),
                    data_dir, 'test_media_labels_{}.npy'.format(s)), raw_media)
            print(raw_images.shape, raw_lumen.shape)
            